**numpy (as np)** for numerical computations

**pandas (as pd)** for data manipulation and analysis

**sklearn** for machine learning algorithms:

**KMeans** for clustering

**neighbors** for nearest neighbors algorithm

**train_test_split** for splitting data into training and testing sets

**MinMaxScaler** for scaling data

In [3]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

 loads the data from a CSV file named 'books.csv' into a Pandas DataFrame df. The "on_bad_lines='skip'" parameter tells Pandas to skip any lines with errors.

In [4]:
#loading data
df = pd.read_csv('/content/books.csv', on_bad_lines='skip')
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


the **describe()** method provides summary statistics for the data.

The **df.head()** and **df.describe()** functions are used to display the first few rows of the data and provide a summary of the data, respectively.

In [5]:
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [6]:
#data pre-processing
df2 = df.copy()

creates a new column **rating_between** in df2 based on the **average_rating** column. The rating_between column is used to categorize the ratings into five bins.

In [7]:
df2.loc[ (df2['average_rating'] >= 0) & (df2['average_rating'] <= 1), 'rating_between'] = "between 0 and 1"
df2.loc[ (df2['average_rating'] > 1) & (df2['average_rating'] <= 2), 'rating_between'] = "between 1 and 2"
df2.loc[ (df2['average_rating'] > 2) & (df2['average_rating'] <= 3), 'rating_between'] = "between 2 and 3"
df2.loc[ (df2['average_rating'] > 3) & (df2['average_rating'] <= 4), 'rating_between'] = "between 3 and 4"
df2.loc[ (df2['average_rating'] > 4) & (df2['average_rating'] <= 5), 'rating_between'] = "between 4 and 5"

This section creates two new DataFrames **rating** and **lang** using the **get_dummies** method from Pandas. The get_dummies method *converts categorical variables into numerical variables*.

The **get_dummies** function is used to create binary columns for each category.

In [8]:
#feature engineering
rating=pd.get_dummies(df2['rating_between'])
lang=pd.get_dummies(df2['language_code'])

The **features** DataFrame is created by concatenating the **rating**, **lang**, **average_rating**, and **ratings_count **columns along the columns **axis (axis=1)**.

In [9]:
features = pd.concat([rating,
                      lang,
                      df2['average_rating'],
                      df2['ratings_count']

], axis=1)

In [10]:
# scaling the data
scaler=MinMaxScaler()

This section creates a **MinMaxScaler** object and uses it to scale the features DataFrame. The **fit_transform** method scales the data to a common range, usually between *0 and 1.*

In [11]:
features=scaler.fit_transform(features)

This section creates a **NearestNeighbors** object and uses it to find the nearest neighbors for each data point in the features DataFrame. The **n_neighbors** parameter specifies the number of nearest neighbors to find, and the algorithm parameter specifies the algorithm to use.

The **fit** method trains the model on the **features DataFrame**, and the **kneighbors** method finds the nearest neighbors for each data point

In [12]:
#nearest neighbours algorithm
model=neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
model.fit(features)
dist, idlist = model.kneighbors(features)

he **df2['title'] == book_title** part creates a boolean mask that selects only the rows where the title column matches the **book_title**. The **.index** attribute returns the index of the selected rows.

If the book was found, this line of code extracts the first (and only) index from the **id_b index**. This is because **id_b** is a pandas Index object, and we need to get the actual index value.

This loop iterates over the indices of the nearest neighbors for the book with index **id_b**. For each neighbor, it appends the title of the book to the **book_list_name** list.

Finally, the function returns the list of recommended books.



In [21]:
#book recommendation function
def get_book_recommendations(book_title):
    id_b = df2[df2['title'] == book_title].index
    if id_b.empty:
        print(f"Book '{book_title}' not found in the dataset.")
        return []
    else:
        id_b = id_b[0]
        book_list_name = []
        for newid in idlist[id_b]:
            book_list_name.append(df2.loc[newid].title)
        return book_list_name

This line of code prompts the user to enter a book title and stores the input in the **book_title** variable.

This line of code calls the **get_book_recommendations** function with the **book_title** as an argument and stores the result in the recommendations variable.

If there are recommendations, this block of code prints a header message and then iterates over the recommendations list using the enumerate function, which returns both the index and the value of each item in the list

In [19]:
def main():
    book_title = input("Enter the book title for recommendation: ")
    recommendations = get_book_recommendations(book_title)
    if recommendations:
        print("Book recommendations:")
        for i, book in enumerate(recommendations):
            print(f"{i+1}. {book}")
    else:
        print("No recommendations found.")

In summary, this function prompts the user to enter a book title, gets the book recommendations using the **get_book_recommendations** function, and prints the recommendations if there are any. If there are no recommendations, it prints a message indicating that no recommendations were found.

In [20]:
if __name__ == "__main__":
    main()


Enter the book title for recommendation: To Kill a Mockingbird
Book recommendations:
1. To Kill a Mockingbird
2. Vampire Chronicles: Interview with the Vampire  The Vampire Lestat  The Queen of the Damned (Anne Rice)
3. Hands of Light: A Guide to Healing Through the Human Energy Field
4. The Lord of the Rings Sketchbook
5. When Christ and His Saints Slept  (Henry II and Eleanor of Aquitaine  #1)
6. There are No Children Here: The Story of Two Boys Growing Up in the Other America


Let's say the user wants to get book recommendations for the following books:

1."Harry Potter and the Half-Blood Prince (Harry Potter #6)"  
2."To Kill a Mockingbird"  
3."The Lord of the Rings"  
4."Pride and Prejudice"  
5."The Hunger Games"